In [1]:
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd, numpy as np
import os
import re

class XML2DataFrame:
    '''
    Transforming zip archive with xml files to dataframe
    '''
    def __init__(self, file):
        self.file = file
        self.nalogs_columns = ["ИННЮЛ", 'НаимОрг',
                               'Единый сельскохозяйственный налог',
                               'Налог на добычу полезных ископаемых', 
                               'Налог на игорный'
                              ,'Регулярные платежи за добычу полезных ископаемых (роялти) при выполнении соглашений о разделе продукции',
                               'Сборы за пользование объектами животного мира и за пользование объектами ВБР'
                             , 'Акцизы, всего']
        self.itog_columns = ["INN","NAME_ORG","ESN", "NDPI","IGORNII",
                                     "REG_NDPI","SBOR_JIV", 'ACSCIZ']
        self.dataframe = pd.DataFrame(columns=['Фамилия', 'Имя', 'ИдДок',
                                               'ДатаДок', 'ДатаСост', 
                                               'НаимОрг', 'ИННЮЛ'])
        self.dataframe_itog = pd.DataFrame(columns = self.itog_columns)
        
  
    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]
  

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            if re.fullmatch("НаимНалог", key):
                key1 = element.attrib.get(key)
            elif re.fullmatch("СумУплНал", key):
                parsed[key1] = element.attrib.get(key)
            else:
                parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed
  
    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)
    
    def process_all_files_in_dir(self):
        
        with ZipFile(self.file) as z:
            for ind, curr_file in enumerate(z.namelist()):
                tree = ET.ElementTree(file = z.open(curr_file))
                self.root = tree.getroot()
                self.dataframe = self.dataframe.append(self.process_data(),
                                                       ignore_index=True)
                try:
                    self.dataframe = self.dataframe\
                                        .loc[:, self.nalogs_columns]

                except KeyError:
                    self.dataframe = self.dataframe\
                         .loc[:, set(self.nalogs_columns)\
                         .intersection(set(self.dataframe.columns))]
                    self.dataframe.loc[:, set(self.nalogs_columns)\
                            .difference(set(self.dataframe.columns))] = np.NaN
                self.dataframe = self.dataframe.loc[:, self.nalogs_columns]\
                    .rename(columns = {
                             "ИННЮЛ":"INN",
                             'НаимОрг':"NAME_ORG",
                             'Единый сельскохозяйственный налог':"ESN",
                             'Налог на добычу полезных ископаемых':"NDPI",
                             'Налог на игорный':"IGORNII",
                             "Регулярные платежи за добычу полезных ископаемых (роялти) при выполнении соглашений о разделе продукции": "REG_NDPI",
                             "Сборы за пользование объектами животного мира и за пользование объектами ВБР":"SBOR_JIV",
                             'Акцизы, всего':'ACSCIZ'})
                self.dataframe = self.dataframe[~self.dataframe\
                                                .loc[:, self.itog_columns[2:]]\
                                                .isna().all(axis=1)]
                self.dataframe_itog = self.dataframe_itog.append(self.dataframe,
                                                                 ignore_index=True)                
                self.dataframe = pd.DataFrame(columns=['Фамилия', 'Имя', 'ИдДок', 'ДатаДок', 
                                           'ДатаСост', 'НаимОрг', 'ИННЮЛ'])
                if ind % 1000 == 0:
                    print("%i) %i" % (ind, self.dataframe_itog.shape[0]))
                    
        #Заменяем нули на нан, так как нас интересуют только уплаченные налоги
        self.dataframe_itog.replace({"0.00":np.NaN}, inplace = True)
        #Создаем табличку для заливки в SQL
        self.nalogi_reestr = np.zeros((0,3))
        self.df_val = self.dataframe_itog.iloc[:, 2:].astype("float").values
        
        for i in range(self.dataframe_itog.shape[0]):
            for j in range(self.df_val.shape[1]):
                if (~np.isnan(self.df_val[i][j])):
                    self.val = np.array([self.dataframe_itog.loc[i,"INN"], 
                                         self.dataframe_itog.columns[2+j], 
                                         0], dtype="object").reshape(-1,3)
                    self.nalogi_reestr = np.vstack((self.nalogi_reestr,
                                                                self.val))

        self.nalogi_reestr = pd.DataFrame(self.nalogi_reestr, 
                                     columns=["INN", "REESTR", 
                                              "FLAG_HARD_REESTR"], dtype=str)
        self.replace_hard = self.nalogi_reestr[self.nalogi_reestr.REESTR 
                                                       == 'IGORNII'].index

        self.nalogi_reestr.loc[self.replace_hard, "FLAG_HARD_REESTR"] = "1"

        return self.nalogi_reestr


In [2]:
# Указываем скачанный архив
arhivchick = XML2DataFrame("data-20201201-structure-20180801.zip")

In [3]:
df = arhivchick.process_all_files_in_dir()

0) 1
1000) 2802
2000) 5575
3000) 8519
4000) 11382
5000) 14181
6000) 17207
7000) 19949
8000) 22700
9000) 25417
10000) 28217
11000) 31112
12000) 33959


In [4]:
df.to_excel('all_nalogs.xlsx', index=False)